In [83]:
import pandas as pd
import numpy as np

# Read the data from the csv file

education = pd.read_csv("../Data Generation/data_wrangling_education_2023_u7322074.csv")
medical = pd.read_csv("../Data Generation/data_wrangling_medical_2023_u7322074.csv")

In [74]:
education['state'] = education['state'].str.lower()

In [41]:
eduDups = education[education["ssn"].duplicated(keep=False)]
medDups = medical[medical["ssn"].duplicated(keep=False)]

In [110]:
eduUnique = education.drop_duplicates(subset=['ssn'], keep=False)
medUnique = medical.drop_duplicates(subset=['ssn'], keep=False)

In [43]:
temp = pd.merge(eduUnique, medUnique, on = ['ssn'], how='inner')

In [77]:
len(education[~education["ssn"].isin(medical["ssn"])])

3110

In [78]:
len(medical[~medical["ssn"].isin(education["ssn"])])

3918

In [55]:
eduAlone = education[~education["ssn"].isin(medical["ssn"])]
medAlone = medical[~medical["ssn"].isin(education["ssn"])]

In [56]:
eduAlone.columns

Index(['rec_id', 'ssn', 'first_name', 'middle_name', 'last_name', 'gender',
       'current_age', 'birth_date', 'street_address', 'suburb', 'postcode',
       'state', 'phone', 'email', 'education', 'occupation', 'salary',
       'credit_card_number', 'years_of_experience', 'employment_timestamp'],
      dtype='object')

In [57]:
medAlone.columns

Index(['rec_id', 'ssn', 'first_name', 'middle_name', 'last_name', 'gender',
       'age_at_consultation', 'birth_date', 'medicare_number',
       'street_address', 'suburb', 'postcode', 'state', 'phone', 'email',
       'marital_status', 'height', 'weight', 'bmi', 'blood_pressure',
       'cholesterol_level', 'smoking_status', 'clinical_notes',
       'consultation_timestamp'],
      dtype='object')

In [82]:
eduAlone = eduAlone[['ssn', 'first_name', 'middle_name', 'last_name', 'gender', 'birth_date', 'postcode', 'phone']]
medAlone = medAlone[['ssn', 'first_name', 'middle_name', 'last_name', 'gender', 'birth_date', 'postcode', 'phone']]

In [85]:
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

def sim(a_file, b_file):
    # count word occurrences
    a_vals = Counter(a_file)
    b_vals = Counter(b_file)

    # convert to word-vectors
    words  = list(a_vals.keys() | b_vals.keys())
    a_vect = [a_vals.get(word, 0) for word in words]       
    b_vect = [b_vals.get(word, 0) for word in words]        

    # find cosine
    len_a  = sum(av*av for av in a_vect) ** 0.5             
    len_b  = sum(bv*bv for bv in b_vect) ** 0.5             
    dot    = sum(av*bv for av,bv in zip(a_vect, b_vect))   
    cosine = dot / (len_a * len_b) 

    return cosine


In [86]:
from tqdm import tqdm

In [99]:
ids = []

In [100]:
for ssn in tqdm(eduDups["ssn"].unique()):
    rows = eduDups[eduDups["ssn"] == ssn]

    comp = medical[medical["ssn"] == ssn]

    rows["temp"] = rows.apply(lambda row: sim(row, comp.iloc[0].values), axis=1)

    ids.append(rows[rows["temp"] == rows["temp"].max()].index[0])


  0%|          | 0/808 [00:00<?, ?it/s]C:\Users\orect\AppData\Local\Temp\ipykernel_43580\3722742203.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows["temp"] = rows.apply(lambda row: sim(row, comp.iloc[0].values), axis=1)
C:\Users\orect\AppData\Local\Temp\ipykernel_43580\3722742203.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows["temp"] = rows.apply(lambda row: sim(row, comp.iloc[0].values), axis=1)
C:\Users\orect\AppData\Local\Temp\ipykernel_43580\3722742203.py:6: SettingWithCopyWarning: 
A

In [103]:
eduDups = eduDups.loc[ids]

In [112]:
eduUnique = pd.concat([eduUnique, eduDups])

In [113]:
eduUnique

,rec_id,ssn,first_name,middle_name,last_name,gender,current_age,birth_date,street_address,suburb,postcode,state,phone,email,education,occupation,salary,credit_card_number,years_of_experience,employment_timestamp
0,R59823679,h129873596,benjamin,james,huskins,m,54,7/5/1966,1 lanyana way noosa professional centre,noosa heads,4567.0,qld,07 9918 6808,benjamin.huskins@mail.com.au,certificate-i,carer,NaN,1540 5607 3358 0597,35,2020-07-22t21:28+00:00
1,R41534250,d152403541,jennifer,lehan,peacock,f,32,2/5/1988,70 arcadia road pooley bridge,arcadia,3631.0,vic,03 9341 7953,jennifer.peacock@mail.com.au,secondary-school,shop-assistent,5183.0,6114 2815 1674 2872,2,2009-10-20t02:56+00:00
2,R05763850,i105875306,amy,jurkovics,martin,f,19,19/1/2001,868 old calder highway keilor retirement...,keilor,3036.0,vic,03 4254 3667,NaN,certificate-ii,other-service-worker,NaN,2148 0654 5539 3759,0,2020-06-24t18:11+00:00
3,R69132047,f142931760,jean,m,zeok,f,50,19/10/1970,301 galston road rowland vlge,galston,2159.0,nsw,02 9839 0762,zeok50@yahoo.com,diploma,other-technician,NaN,5880 1891 8543 0411,31,2020-06-17t12:43+00:00
4,R79412591,a149195172,geoffrey,kirby,powell,f,47,23/4/1973,3 ibis road st johns terrace,wangaratta,3677.0,vic,03 5321 2414,powell16@mail.com,masters-degree,chief-executive,156775.0,0146 8034 1699 7228,28,2020-07-24t12:01+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18472,R37584986,c183876495,anthony,lewis,berenson,m,34,20/1/1986,22 pedder street nelumie flats,launceston south,7249.0,tas,03 0789 5664,berenson24@yahoo.com,certificate-iii,other-worker,95344.0,5567 0130 3253 6854,15,2020-06-18t21:13+00:00
18955,R72674753,g135627747,jason,eugene,kazura,f,68,7/6/1948,16 Towerhill Road,Frankston,3199.0,vic,03 9607 4640,kazura29@yahoo.com,secondary-school,farm-worker,-9999.0,3049 4420 3538 4076,53,2016-09-14T09:18+00:00
18779,R75952468,g195286574,tony,thomas,turner,f,27,24/6/1989,17 Levien Street,Scullin,2614.0,act,NaN,turner.tony@yahoo.com,advance-diploma,health-professional,NaN,8116 2648 1462 0983,10,2014-10-19T13:16+00:00
19841,R12603334,h124363301,diana,miller,smith,f,57,17/6/1963,1 townsend road edeanlea,buderim,4556.0,qld,07 9289 1877,smith16@yahoo.com,certificate-i,secretary,34253.0,0489 4273 6414 2577,35,2017-07-09t03:47+00:00


In [114]:
temp = pd.merge(eduUnique, medUnique, on = ['ssn'], how='inner')

In [121]:
eduUnique.to_csv("../Data Generation/education.csv", index=False)

In [116]:
temp.reset_index(drop=True, inplace=True)

In [118]:
temp

,rec_id_x,ssn,first_name_x,middle_name_x,last_name_x,gender_x,current_age,birth_date_x,street_address_x,suburb_x,...,email_y,marital_status,height,weight,bmi,blood_pressure,cholesterol_level,smoking_status,clinical_notes,consultation_timestamp
0,R59823679,h129873596,benjamin,james,huskins,m,54,7/5/1966,1 lanyana way noosa professional centre,noosa heads,...,benjamin.huskins@mail.com.au,not-married,178,133,41,77,148,0,compression-fracture-at-l1,2012-09-01t16:35+00:00
1,R41534250,d152403541,jennifer,lehan,peacock,f,32,2/5/1988,70 arcadia road pooley bridge,arcadia,...,jennifer.peacock@mail.com.au,married,167,124,44,81,122,0,possible-food-poisoning,2015-11-06t16:02+00:00
2,R05763850,i105875306,amy,jurkovics,martin,f,19,19/1/2001,868 old calder highway keilor retirement...,keilor,...,amy71.mail.com.au,divorced,173,69,23,77,148,0,worsening-hypercholesterolemia,2015-03-15t14:01+00:00
3,R69132047,f142931760,jean,m,zeok,f,50,19/10/1970,301 galston road rowland vlge,galston,...,NaN,separated,165,138,50,72,176,0,low-energy,2014-06-04t02:37+00:00
4,R79412591,a149195172,geoffrey,kirby,powell,f,47,23/4/1973,3 ibis road st johns terrace,wangaratta,...,powell16@mail.com,separated,185,68,19,79,180,0,abdominal-pain-left-side,2014-09-23t10:40+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16077,R37584986,c183876495,anthony,lewis,berenson,m,34,20/1/1986,22 pedder street nelumie flats,launceston south,...,berenson24@yahoo.com,married,195,85,22,74,112,0,fall,2017-03-10t01:38+00:00
16078,R72674753,g135627747,jason,eugene,kazura,f,68,7/6/1948,16 Towerhill Road,Frankston,...,kazura29@yahoo.com,married,173,60,20,71,119,0,oral-fungus-(inside-lips),2013-11-27t11:14+00:00
16079,R75952468,g195286574,tony,thomas,turner,f,27,24/6/1989,17 Levien Street,Scullin,...,turner.tony@yahoo.com,married,196,55,14,76,198,1,musculoskeletal-pain,2014-03-24t09:25+00:00
16080,R12603334,h124363301,diana,miller,smith,f,57,17/6/1963,1 townsend road edeanlea,buderim,...,NaN,married,172,132,44,73,263,0,hypertension,2014-06-26t12:12+00:00


In [119]:
temp.to_csv("../Data Generation/data_wrangling_merged_2023_u7322074.csv", index=False)

In [122]:
education.columns

Index(['rec_id', 'ssn', 'first_name', 'middle_name', 'last_name', 'gender',
       'current_age', 'birth_date', 'street_address', 'suburb', 'postcode',
       'state', 'phone', 'email', 'education', 'occupation', 'salary',
       'credit_card_number', 'years_of_experience', 'employment_timestamp'],
      dtype='object')

In [123]:
medical.columns

Index(['rec_id', 'ssn', 'first_name', 'middle_name', 'last_name', 'gender',
       'age_at_consultation', 'birth_date', 'medicare_number',
       'street_address', 'suburb', 'postcode', 'state', 'phone', 'email',
       'marital_status', 'height', 'weight', 'bmi', 'blood_pressure',
       'cholesterol_level', 'smoking_status', 'clinical_notes',
       'consultation_timestamp'],
      dtype='object')

In [133]:
education = pd.read_csv("../Data Generation/education.csv")
medical = pd.read_csv("../Data Generation/data_wrangling_medical_2023_u7322074.csv")

cols = {'first_name' : 0, 'middle_name' : 0, 'last_name' : 0, 'gender' : 0, 'birth_date' : 0, 'street_address' : 0, 'suburb' : 0, 'postcode' : 0, 'state' : 0, 'phone' : 0, 'email' : 0}

In [134]:
merged = pd.read_csv("../Data Generation/data_wrangling_merged_2023_u7322074.csv")

In [135]:
for i in tqdm(range(len(merged))):
    row = merged.iloc[i]
    for col in cols.keys():
        if row[col + "_x"] != row[col + "_y"]:
            cols[col] += 1

            if row[col + "_y"] != np.nan:
                merged.loc[i, col + "_x"] = row[col + "_y"]

100%|██████████| 16082/16082 [00:12<00:00, 1273.16it/s]


In [132]:
cols

{'first_name': 2,
 'middle_name': 6046,
 'last_name': 8,
 'gender': 3166,
 'birth_date': 0,
 'street_address': 13214,
 'suburb': 12990,
 'postcode': 16920,
 'state': 5308,
 'phone': 18277,
 'email': 14832}

In [137]:
merged.to_csv("../Data Generation/data_wrangling_merged_2023_u7322074.csv", index=False)

In [174]:
merged.describe()

,current_age,postcode_x,salary,years_of_experience,age_at_consultation,postcode_y,height,weight,bmi,blood_pressure,cholesterol_level,smoking_status
count,16082.000000,12819.000000,13613.000000,16082.000000,16082.000000,12819.000000,16082.000000,16082.000000,16082.000000,16082.000000,16082.000000,16082.000000
mean,42.318679,3453.340666,57167.304856,22.656946,37.773038,3453.340666,175.067591,72.598433,28.983584,74.864756,169.867181,0.133379
std,19.706345,1592.886188,55356.444612,17.640680,19.625053,1592.886188,19.303611,65.462672,9.093985,3.994001,51.119550,0.339994
min,5.000000,800.000000,-9999.000000,0.000000,2.000000,800.000000,80.000000,-99.000000,11.000000,60.000000,0.000000,0.000000
25%,25.000000,2430.000000,0.000000,5.000000,20.000000,2430.000000,169.000000,60.000000,21.000000,72.000000,135.000000,0.000000
50%,42.000000,3150.000000,56239.000000,21.000000,37.000000,3150.000000,178.000000,85.000000,28.000000,75.000000,170.000000,0.000000
75%,59.000000,4370.000000,100930.000000,39.000000,55.000000,4370.000000,189.000000,117.000000,36.000000,78.000000,204.000000,0.000000
max,96.000000,7331.000000,199991.000000,56.000000,91.000000,7331.000000,199.000000,149.000000,54.000000,92.000000,395.000000,1.000000


In [176]:
merged["weight"].value_counts()

weight
-99    1626
 64     183
 73     181
 72     179
 61     179
       ... 
 16      17
 17      14
 15       5
 14       1
 10       1
Name: count, Length: 138, dtype: int64

In [183]:
red = [i for i in merged.columns if i.endswith("_y") or i.endswith("_x")]

In [184]:
red.remove("rec_id_y")
red.remove("rec_id_x")

In [189]:
columns = [k[:-2] for k in red[:11]]

In [190]:
columns

['first_name',
 'middle_name',
 'last_name',
 'gender',
 'birth_date',
 'street_address',
 'suburb',
 'postcode',
 'state',
 'phone',
 'email']

In [191]:
def mg(vals):
    if vals[0] != np.nan and vals[1] != np.nan:
        if vals[0] == vals[1]:
            return vals[0]
        else:
            return vals[1]
    elif vals[0] != np.nan:
        return vals[0]
    elif vals[1] != np.nan:
        return vals[1]
    else:
        return np.nan

In [193]:
for col in columns:
    merged[col] = merged[[col + "_x", col + "_y"]].values.tolist()

    merged[col] = merged[col].apply(lambda x: mg(x))

merged.drop(red, axis=1, inplace=True)

In [195]:
merged.columns

Index(['rec_id_x', 'ssn', 'current_age', 'education', 'occupation', 'salary',
       'credit_card_number', 'years_of_experience', 'employment_timestamp',
       'rec_id_y', 'age_at_consultation', 'medicare_number', 'marital_status',
       'height', 'weight', 'bmi', 'blood_pressure', 'cholesterol_level',
       'smoking_status', 'clinical_notes', 'consultation_timestamp',
       'first_name', 'middle_name', 'last_name', 'gender', 'birth_date',
       'street_address', 'suburb', 'postcode', 'state', 'phone', 'email'],
      dtype='object')

In [197]:
merged["birth_date"] = pd.to_datetime(merged["birth_date"], infer_datetime_format=True, errors='coerce')

C:\Users\orect\AppData\Local\Temp\ipykernel_43580\3196212174.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  merged["birth_date"] = pd.to_datetime(merged["birth_date"], infer_datetime_format=True, errors='coerce')


In [203]:
import math

In [212]:
now = pd.Timestamp('now')
merged["current_age"] = now - merged["birth_date"]

merged["current_age"] = (merged["current_age"]/365).dt.days

In [213]:
merged["current_age"].describe()

count    6356.000000
mean       45.274544
std        19.564937
min         7.000000
25%        28.000000
50%        46.000000
75%        62.000000
max        99.000000
Name: current_age, dtype: float64

In [214]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16082 entries, 0 to 16081
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   rec_id_x                16082 non-null  object        
 1   ssn                     16082 non-null  object        
 2   current_age             6356 non-null   float64       
 3   education               16082 non-null  object        
 4   occupation              14792 non-null  object        
 5   salary                  13613 non-null  float64       
 6   credit_card_number      14792 non-null  object        
 7   years_of_experience     16082 non-null  int64         
 8   employment_timestamp    16082 non-null  object        
 9   rec_id_y                16082 non-null  object        
 10  age_at_consultation     16082 non-null  int64         
 11  medicare_number         16082 non-null  object        
 12  marital_status          14063 non-null  object

In [215]:
merged["first_name"] = merged["first_name"].str.capitalize()
merged["middle_name"] = merged["middle_name"].str.capitalize()
merged["last_name"] = merged["last_name"].str.capitalize()
merged["street_address"] = merged["street_address"].str.capitalize()
merged["suburb"] = merged["suburb"].str.capitalize()
merged["state"] = merged["state"].str.upper()

In [218]:
merged["clinical_notes"].replace("-", " ", inplace=True)

In [219]:
merged.to_csv("../Data Generation/data_wrangling_merged_2023_u7322074.csv", index=False)